In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import map_coordinates
from scipy.interpolate import interp1d
import sys
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm
import plotly.graph_objects as go
from scipy.special import lpmv

In [2]:
class Legendre:

    def __init__(self,lm_max,l_max,m_max,n_theta_max,n_phi_max):
        #input parameters
        self.lm_max=lm_max#total number of lm pairs
        self.l_max=l_max#maximum degree
        self.m_max=m_max#maximum order
        self.n_theta_max=n_theta_max#maximum grid points in theta direction
        self.n_phi_max=n_phi_max#maximum grid points in phi direction

        #arrays to be initialized
        self.theta=None #theta grid
        self.weights=None#integration weights
        self.ylm2lm=None#mapping (l,m)->packed index
        self.lm2l=None#unpack l
        self.lm2m=None#unpack m

        #derived quantities
        self.n_blocks=1
        self.blokcs=None

        #Step 1: Gauss-Legendre Quadrature
    def gauleg(self):
        x,w=np.polynomial.legendre.leggauss(self.n_theta_max)
        self.theta=np.arccos(x)
        self.weights=w

        #Step2: Build (l,m) index mapping
    def build_index_mapping(self):
        lm=0
        self.ylm2lm={}
        self.lm2l=np.zeros(self.lm_max,dtype=int)
        self.lm2m=np.zeros(self.lm_max,dtype=int)

        for l in range(self.l_max+1):
            for m in range(-min(l,self.m_max),min(l,self.m_max)+1):
                if lm>=self.lm_max:
                    break
                self.ylm2lm[(l,m)]=lm
                self.lm2l[lm]=l
                self.lm2m[lm]=m
                lm+=1

        #Step3: Block structure
    def get_blocks(self):
        n_blocks=self.n_blocks
        lmax,mmax=self.l_max,self.m_max
        all_pairs=[(l,m) for l in range(lmax+1) for m in range(min(mmax,l)+1)]
        blocks=[[] for _ in range(n_blocks)]
        for i,pair in enumerate(all_pairs):
            blocks[i%n_blocks].append(pair)
        self.n_blocks=n_blocks
        self.blocks=blocks
        print(blocks)
        #return blocks

    #compute the plm and dplm
    def compute_plm_derivatives(self):

        ntheta=self.n_theta_max
        lmax=self.l_max
        mmax=self.m_max
        theta=self.theta
        x=np.cos(theta)
        sin_th=np.sin(theta)

        #allocate
        Plm=np.zeros((lmax+1,mmax+1,ntheta))
        dPlm=np.zeros_like(Plm)

        #loop over m
        for m in range(mmax+1):
            #initlal Pm^m
            fact=1.0
            for k in range(1,2*m,2):
                fact*=k
            Plm[m,m,:]=((-1)**m)*fact*(sin_th**m)

            if m<lmax:
                Plm[m+1,m,:]=(2*m+1)*x*Plm[m,m,:]
            #recurrence for higher l
            for l in range(m+2,lmax+1):
                Plm[l,m,:]=((2*l-1)*x*Plm[l-1,m,:]-(l+m-1)*Plm[l-2,m,:])/(l-m)
        #derivatives using recurrence relation

        for l in range(1,lmax+1):
            for  m in range(0,min(l,mmax)+1):
                dPlm[l,m,:]=(l*x*Plm[l,m,:]-(l+m)*Plm[l-1,m,:])/(-sin_th)
        return Plm,dPlm
        

In [3]:
class Legendre_1():
    def __init__(self,l_max,minc,n_theta_max,m_max):
        
        #maximum degrees and order
        #self.lm_max=None
        self.m_max=m_max
        self.l_max=l_max
        self.minc=minc
        self.n_m_max=m_max//minc+1
        self.n_theta_max=n_theta_max
        self.n_phi_max=n_theta_max*2//2

        #now intialize the lm_max
        self.lm_max=0
        for m in range(0,m_max+1,minc):
            for l in range(m,l_max+1):
                self.lm_max+=1
        
        #arrays
        self.Plm=np.zeros((self.lm_max,n_theta_max//2))    #P_l^m(cos(theta))
        self.wPlm=np.zeros_like(self.Plm)   #weighted P_l^m
        self.wdPlm=np.zeros_like(self.Plm)  #weighted derivative
        self.dPlm=np.zeros_like(self.Plm)   #derivative wrt theta
        self.dPhi=np.zeros_like(self.Plm)   #derivative wrt phi
        self.sinTh=np.zeros(n_theta_max)  #sin(theta) values
        self.gauss=np.zeros(n_theta_max)  #gauss-legendre polynomial

        #indexing arrays
        self.lStart=np.zeros(self.n_m_max,dtype=int)
        self.lStop=np.zeros(self.n_m_max,dtype=int)
        self.lmOdd=np.zeros(self.n_m_max,dtype=bool) #odd even flag

        #store temporary varables
        self.plma=np.zeros(self.lm_max)
        self.dtheta_plma=np.zeros(self.lm_max)
        #compute nodes and weight
        self.theta_ord,self.gauss=self._gauleg(n_theta_max)
        self.sinTh=np.sin(self.theta_ord)

        #compute Plm,dPlm,wPlm,wdPlm,dphi
        dpi=np.pi
        for n_theta in range(n_theta_max//2):
            #colat=self.sinTh[n_theta]
            #call placeholder pl_theta(to be implemented)
            colat=self.theta_ord[n_theta]
            #self.plma,self.dtheta_plma=self._plma_theta(colat,self.l_max,self.m_max,self.minc,self.lm_max)
            plma,dtheta_plma=self._plma_theta(colat,self.l_max,self.m_max,self.minc,self.lm_max)

            #lm=-1
            lm=0
            for m in range(0,m_max+1,minc):
                for l in range(m,l_max+1):
                    lm+=1
                    phase=(-1.0)**m
                    #self.Plm[lm-1,n_theta]=phase*self.plma[lm-1]
                    self.Plm[lm-1,n_theta]=phase*plma[lm-1]
                    #true theta derivative
                    #self.dPlm[lm-1,n_theta]=phase*self.dtheta_plma[lm-1]/np.sin(colat)
                    self.dPlm[lm-1,n_theta]=phase*dtheta_plma[lm-1]/np.sin(colat)
                    #add theta dependence in dphi
                    #self.dPhi[lm-1,n_theta]=float(m)/np.sin(colat)
                    self.dPhi[lm-1,n_theta]=float(m)/np.sin(colat)
                    #weighted versions
                    self.wPlm[lm-1,n_theta]=2.0*dpi*self.gauss[n_theta]*self.Plm[lm-1,n_theta]
                    self.wdPlm[lm-1,n_theta]=2.0*dpi*self.gauss[n_theta]*self.dPlm[lm-1,n_theta]

    def _getblocs(self,l_max,minc,n_m_max):
        lStart=np.zeros(n_m_max,dtype=int)
        lStop=np.zeros(n_m_max,dtype=int)
        lmOdd=np.zeros(n_m_max,dtype=int)

        lStart[0]=1
        lStop[0]=l_max+1
        lmOdd[0]=(l_max%2==0)

        for mc in range(1,n_m_max):
            m=mc*minc
            lStart[mc]=lStop[mc-1]+1
            lStop[mc]=lStart[mc]+(l_max-m)
            lmOdd[mc]=((lStop[mc]-lStart[mc])%2==0)
        
        return lStart,lStop,lmOdd

    def _gauleg(self,n_theta_max):
        "compute gauss-legendre nodes and weight"
        theta_ord=np.zeros(n_theta_max)
        gauss=np.zeros(n_theta_max)
        
        dpi=np.pi
        M=(n_theta_max+1)//2
        XXM=0.0
        XXL=1.0
        eps=3e-14

        for i in range(1,M+1):
            zz=np.cos(dpi*((i-0.25)/(n_theta_max+0.5)))
            zz1=0.0

            while abs(zz-zz1)>eps:
                p1=1.0
                p2=0.0
                for j in range(1,n_theta_max+1):
                    p3=p2
                    p2=p1
                    p1=((2*j-1)*zz*p2-(j-1)*p3)/j
                pp=n_theta_max*(zz*p1-p2)/(zz*zz-1.0)
                zz1=zz
                zz=zz1-p1/pp

            theta_ord[i-1]=np.arccos(XXM+XXL*zz)
            theta_ord[n_theta_max-i]=np.arccos(XXM-XXL*zz)
            w=2.0*XXL/((1.0-zz**2)*pp**2)
            gauss[i-1]=w
            gauss[n_theta_max-i]=w
            
        return theta_ord,gauss
        
    def _plma_theta(self,theta,max_degree,max_order,m0,ndim_plma):
        plma=np.zeros(ndim_plma)
        dtheta_plma=np.zeros(ndim_plma)

        dnorm=1.0/(2.0*np.sqrt(np.pi))

        pos=0

        sin_theta=np.sin(theta)
        cos_theta=np.cos(theta)

        #first pass: compute plma and one extra plm0 used for dtheta_plma last element
        for m in range(0,max_order+1,m0):
            #compute fac(double factorial related product)
            fac=1.0
            #j runs 3,5,7,......,2*m+1 inclusive when m>=1
            if m>=1:
                for j in range(3,2*m+2,2):
                    fac*=float(j)/float(j-1)

            plm0=np.sqrt(fac)

            if abs(sin_theta)>0.0:
                plm0=plm0*((-sin_theta)**m)
            elif m!=0:
                plm0=0.0
            #l=m
            plma[pos]=dnorm*plm0
            plm1=0.0

            #higher l recursion
            for l in range(m+1,max_degree+1):
                plm2=plm1
                plm1=plm0
                #coefficients under square roots
                a=((2*l-1)*(2*l+1))/float((l-m)*(l+m))
                b=((2*l+1)*(l+m-1)*(l-m-1))/float((2*l-3)*(l-m)*(l+m))

                #protect small negatives due to roundoff
                a=max(a,0.0)
                b=max(b,0.0)
                plm0=(cos_theta*np.sqrt(a)*plm1-np.sqrt(b)*plm2)
                pos+=1
                if pos-1<ndim_plma:
                    plma[pos-1]=dnorm*plm0
                else:
                    pass

            if max_degree>=m:
                #use l=max_degree +1 with current plm1,plm2
                l=max_degree+1
                #compute new plm0 as in fortran
                a=((2*l-1)*(2*l+1))/float((l-m)*(l+m))
                b=((2*l+1)*(l+m-1)*(l-m-1))/float((2*l-3)*(l-m)*(l+m))
                a=max(a,0.0)
                b=max(b,0.0)

                plm0=(cos_theta*np.sqrt(a)*plm1-np.sqrt(b)*plm2)

                if pos-1>=0 and pos-1<ndim_plma:
                    dtheta_plma[pos-1]=dnorm*plm0

        pos=0
        #second pass:compute dtheta_plma via recurrence relations
        for m in range(0,max_order+1,m0):
            #l=m
            if m<max_degree:
                idx_next=pos+1
                if idx_next<ndim_plma:
                    dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*plma[idx_next]
                else:
                    dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*dtheta_plma[pos]
            else:
                #m==max degree case
                dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*dtheta_plma[pos]

            #interior loop: l=m+1....max_degree-1
            for l in range(m+1,max_degree):
                pos+=1
                idx_plus=pos+1
                idx_minus=pos-1
                term1=l*np.sqrt(((l+m+1.0)*(l-m+1.0))/((2.0*l+1.0)*(2.0*l+3.0)))*plma[pos+1]
                term2=(l+1.0)*np.sqrt(((l+m)*(l-m))/((2.0*l-1.0)*(2.0*l+1.0)))*plma[pos-1]
                dtheta_plma[pos]=term1-term2

            if m<max_degree:
                pos+=1
                l=max_degree
                term1=l*np.sqrt(((l+m+1.0)*(l-m+1.0))/((2.0*l+1.0)*(2.0*l+3.0)))*dtheta_plma[pos]
                term2=(l+1.0)*np.sqrt(((l+m)*(l-m))/((2.0*l-1.0)*(2.0*l+1.0)))*plma[pos-1]
                dtheta_plma[pos]=term1-term2

            pos+=1

        return plma,dtheta_plma




In [4]:
class Legendre_2():
    def __init__(self,l_max,minc,n_theta_max,m_max):
        
    def _gauleg(self,n_theta_max):
        "compute gauss-legendre nodes and weight"
        theta_ord=np.zeros(n_theta_max)
        gauss=np.zeros(n_theta_max)
        
        dpi=np.pi
        M=(n_theta_max+1)//2
        XXM=0.0
        XXL=1.0
        eps=3e-14

        for i in range(1,M+1):
            zz=np.cos(dpi*((i-0.25)/(n_theta_max+0.5)))
            zz1=0.0

            while abs(zz-zz1)>eps:
                p1=1.0
                p2=0.0
                for j in range(1,n_theta_max+1):
                    p3=p2
                    p2=p1
                    p1=((2*j-1)*zz*p2-(j-1)*p3)/j
                pp=n_theta_max*(zz*p1-p2)/(zz*zz-1.0)
                zz1=zz
                zz=zz1-p1/pp

            theta_ord[i-1]=np.arccos(XXM+XXL*zz)
            theta_ord[n_theta_max-i]=np.arccos(XXM-XXL*zz)
            w=2.0*XXL/((1.0-zz**2)*pp**2)
            gauss[i-1]=w
            gauss[n_theta_max-i]=w
            
        return theta_ord,gauss

    


IndentationError: expected an indented block after function definition on line 2 (3758954227.py, line 4)

In [ ]:
lm_max,l_max,m_max,n_theta_max,n_phi_max=10,3,3,15,8
leg=Legendre(lm_max,l_max,m_max,n_theta_max,n_phi_max)
leg.gauleg()
print("Theta:",leg.theta)
print("Weights:",leg.weights)
leg.build_index_mapping()
print("ylm2lm:",leg.ylm2lm)
leg.get_blocks()
print("Number of blocks:",leg.n_blocks)

In [ ]:
Plm,dPlm=leg.compute_plm_derivatives()
print("Plm:",Plm[2,0,:])
print("dPlm:",dPlm[2,0,:])

In [ ]:
th=np.linspace(0,np.pi,n_theta_max)
x=np.cos(th)
plt.plot(x,Plm[2,0,:])
plt.plot(x,dPlm[2,0,:])

In [ ]:
if __name__=="__main__":
    n_theta=250
    leg=Legendre_1(l_max=4,minc=1,n_theta_max=n_theta,m_max=4)

    print("Theta (colatitude) values:")
    print(leg.theta_ord)

    print("\nGauss weights:")
    print(leg.gauss)

    print("\nSum of weights:",np.sum(leg.gauss))

    #check shapes of all the arrays
    print("Plm:",leg.Plm.shape)
    print("dPlm:",leg.dPlm.shape)
    print("wPlm:",leg.wPlm.shape)
    print("wdPlm:",leg.wdPlm.shape)
    print("dPhi:",leg.dPhi.shape)
    print("plma:",leg.plma.shape)
    print("dtheta_plma:",leg.dtheta_plma.shape)

    print("\n First few entries of Plm(placeholder)")
    print(leg.Plm.shape)
    print("\n First few entries of dPlm((placeholder)")
    print(leg.dPlm.shape)
    

    #print("The Plm:",leg.Plm)
    #print("\n The dPlm:",leg.dPlm)
    #print("\n The wPlm:",leg.wPlm)
    #print("\n The wdPlm:",leg.wdPlm)
        

In [ ]:
leg2=Legendre_1(l_max=3,minc=1,n_theta_max=4,m_max=3)
x=np.cos(leg2.theta_ord)
w=leg2.gauss

for k in [0,2,4,6]:
    exact=2.0/(k+1) if k%2==0 else 0.0
    approx=np.sum(w*x**k)
    print(f"k={k}: exact={exact:.6e}, approx={approx:.6e}, error={abs(exact-approx):.2e}")

In [ ]:
from math import factorial
l_max=5
m_max=5
minc=1
n_theta_max=500
leg=Legendre_1(l_max,minc,n_theta_max,m_max)

def lm_index(l,m,minc=1):
    idx=0
    for mm in range(0,m_max+1,minc):
        for ll in range(mm,l_max+1):
            if ll==l and mm==m:
                return idx
            idx+=1
    return None

pairs=[(2,0)]#,(4,2),(5,0)]
theta=leg.theta_ord[:n_theta_max//2]
plt.figure(figsize=(10,6))

for l,m in pairs:
    idx=lm_index(l,m,minc)
    plm_fortran=leg.Plm[idx,:]

    raw=lpmv(0,2,np.cos(theta))
    #raw*=(-1)**m
    norm=1/np.sqrt(4*np.pi)
    norm*=np.sqrt((2*l+1)*(factorial(l-m)/factorial(l+m)))
    plm_scipy=norm*raw
    #plm_scipy=lpmv(m,l,np.cos(theta))
    #plm_scipy=(-1)**m*plm_scipy

    plt.plot(theta,plm_fortran,label=f'Fortran-style P_{l}^{m}')
    plt.plot(theta,plm_scipy,'--',label=f'SciPy P_{l}^{m}')
    

In [ ]:
l_max,m_max,minc,n_theta_max=4,4,1,8
leg4=Legendre_1(l_max,minc,n_theta_max,m_max)

theta=np.pi/4
x=np.cos(theta)
dnorm=1.0/(2.0*np.sqrt(np.pi))
print("theta (rad) =",theta)
print("cos (theta) =",x)
print("dnorm =",dnorm)

plma,dtheta_plma=leg4._plma_theta(theta,l_max,m_max,minc,leg4.lm_max)
print("\n plma (first 10 entries)")
for i,v in enumerate(plma[:10]):
    print(i,v)

expected_p00=dnorm*1.0
print("\n expected plma for (l=0,m=0) =",expected_p00)

def scipy_normalized(l,m,theta):
    x=np.cos(theta)
    raw=lpmv(m,l,x)
    norm=np.sqrt((2*l+1)/(4*np.pi)*factorial(l-m)/factorial(l+m))
    return norm*raw

def lm_index(l,m,l_max,minc=1):
    idx=0
    for mm in range(0,m+1,minc):
        for ll in range(mm,l_max+1):
            if(ll,mm)==(l,m):
                return idx
            idx+=1
    raise ValueError("not found")

pairs=[(0,0),(1,0),(1,1),(2,1)]
print("\n Comparisons (Fortran-plma entry vs Scipy-normalized)")
for (ll,mm) in pairs:
    idx=lm_index(ll,mm,l_max,minc)
    fortran_val=plma[idx]
    scipy_val=scipy_normalized(ll,mm,theta)
    print(f"l={ll}, m={mm}, idx={idx}: fortran(plma)={fortran_val:.8e}, scipy_norm={scipy_val:.8e}, ratio={fortran_val/(scipy_val+1e-300):.6e}")

In [ ]:
    #def _plma_theta(self,theta,max_degree,max_order,m0,ndim_plma):
        #plma=np.zeros(ndim_plma)
        #dtheta_plma=np.zeros(ndim_plma)

        #dnorm=1.0/(2.0*np.sqrt(np.pi))

        #pos=0

        #sin_theta=np.sin(theta)
        #cos_theta=np.cos(theta)

        #first pass: compute plma and one extra plm0 used for dtheta_plma last element
        #for m in range(0,max_order+1,m0):
            #compute fac(double factorial related product)
            #fac=1.0
            #j runs 3,5,7,......,2*m+1 inclusive when m>=1
            #if m>=1:
                #for j in range(3,2*m+2,2):
                    #fac*=float(j)/float(j-1)

            #plm0=np.sqrt(fac)

            #if abs(sin_theta)>0.0:
                #plm0=plm0*((-sin_theta)**m)
            #elif m!=0:
                #plm0=0.0
            #l=m
            #plma[pos]=dnorm*plm0
            #plm1=0.0

            #higher l recursion
            #for l in range(m+1,max_degree+1):
                #plm2=plm1
                #plm1=plm0
                #coefficients under square roots
                #a=((2*l-1)*(2*l+1))/float((l-m)*(l+m))
                #b=((2*l+1)*(l+m-1)*(l-m-1))/float((2*l-3)*(l-m)*(l+m))

                #protect small negatives due to roundoff
                #a=max(a,0.0)
                #b=max(b,0.0)
                #plm0=(cos_theta*np.sqrt(a)*plm1-np.sqrt(b)*plm2)
                #pos+=1
                #if pos-1<ndim_plma:
                    #plma[pos-1]=dnorm*plm0
                #else:
                    #pass

            #if max_degree>=m:
                #use l=max_degree +1 with current plm1,plm2
                #l=max_degree+1
                #compute new plm0 as in fortran
                #a=((2*l-1)*(2*l+1))/float((l-m)*(l+m))
                #b=((2*l+1)*(l+m-1)*(l-m-1))/float((2*l-3)*(l-m)*(l+m))
                #a=max(a,0.0)
                #b=max(b,0.0)

                #plm0=(cos_theta*np.sqrt(a)*plm1-np.sqrt(b)*plm2)

                #if pos-1>=0 and pos-1<ndim_plma:
                    #dtheta_plma[pos-1]=dnorm*plm0

        #pos=0
        #second pass:compute dtheta_plma via recurrence relations
        #for m in range(0,max_order+1,m0):
            #l=m
            #if m<max_degree:
                #idx_next=pos+1
                #if idx_next<ndim_plma:
                    #dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*plma[idx_next]
                #else:
                    #dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*dtheta_plma[pos]
            #else:
                #m==max degree case
                #dtheta_plma[pos]=l/np.sqrt(2.0*l+3.0)*dtheta_plma[pos]

            #interior loop: l=m+1....max_degree-1
            #for l in range(m+1,max_degree):
                #pos+=1
                #idx_plus=pos+1
                #idx_minus=pos-1
                #term1=l*np.sqrt(((l+m+1.0)*(l-m+1.0))/((2.0*l+1.0)*(2.0*l+3.0)))*plma[pos+1]
                #term2=(l+1.0)*np.sqrt(((l+m)*(l-m))/((2.0*l-1.0)*(2.0*l+1.0)))*plma[pos-1]
                #dtheta_plma[pos]=term1-term2

            #if m<max_degree:
                #pos+=1
                #l=max_degree
                #term1=l*np.sqrt(((l+m+1.0)*(l-m+1.0))/((2.0*l+1.0)*(2.0*l+3.0)))*dtheta_plma[pos]
                #term2=(l+1.0)*np.sqrt(((l+m)*(l-m))/((2.0*l-1.0)*(2.0*l+1.0)))*plma[pos-1]
                #dtheta_plma[pos]=term1-term2

            #pos+=1

        #return plma,dtheta_plma

            

        
    #def _plma_theta(self,theta,l_max,m_max,minc,lm_max):
        #x=np.cos(theta)
        #plma=np.zeros(lm_max)
        #dtheta_plma=np.zeros(lm_max)

        #lm=0
        #for m in range(0,m_max+1,minc):
            #initial condition P_m^m(x)
            #if m==0:
                #p_mm=1.0
            #else:
                #p_mm=(-1)**m*np.prod([(2*k-1) for k in range(1,m+1)])*(1-x**2)**(m/2)
            #P_{m+1}^m(x)
            #p_mmp1=(2*m+1)*x*p_mm if m<l_max else 0.0

            #store P_m^m
            #lm+=1
            #plma[lm-1]=p_mm
            #if abs(np.sin(theta))>1e-14:
                #dpdx=(m*x*p_mm-(m+m)*0)/(x**2-1) if m>0 else 0
                #dtheta_plma[lm-1]=-np.sin(theta)*dpdx
            #else:
                #dtheta_plma[lm-1]=0.0

            #higher l
            #p_lm2=p_mm
            #p_lm1=p_mmp1
            #for l in range(m+1,l_max+1):
                #p_lm=((2*l-1)*x*p_lm1-(l+m-1)*p_lm2)/l
                #lm+=1
                #plma[lm-1]=p_lm
                #if abs(np.sin(theta))>1e-14:
                    #dpdx=(l*x*p_lm-(l+m)*p_lm1)/(x**2-1)
                    #dtheta_plma[lm-1]=-np.sin(theta)*dpdx
                #else:
                    #dtheta_plma[lm-1]=0.0
                #p_lm2,p_lm1=p_lm1,p_lm
        #plma=np.zeros(self.lm_max)
        #dtheta_plma=np.zeros(self.lm_max)
        #return plma,dtheta_plma